#Import

In [1]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 4.55 ms (started: 2022-05-31 08:24:35 +00:00)


In [2]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 15.9 s (started: 2022-05-31 08:24:35 +00:00)


In [3]:
#setting path to local module
import sys
sys.path.append("/content/drive/MyDrive/Python_NLP")

time: 1.32 ms (started: 2022-05-31 08:24:51 +00:00)


In [4]:
#how to install clone of Russian-adapted branch of ipapy from github to local environment
  #change working directory to that where you want to clone
    ###path_clones = "/content/drive/MyDrive/Python_NLP/Colab_clones"
    ##os.chdir(path_clones)
  #permanently import clone to your directory
    ##!git clone -b eugenefedor-patch-2 --single-branch https://github.com/eugenefedor/ipapy_eugene.git
  #add path to you directory with clone to sys.path for Colab to be able to find module
  #path should be added with the name of inside folder "ipapy_eugene"
    ##sys.path.append("/content/drive/MyDrive/Python_NLP/Colab_clones/ipapy_eugene")
  #once installed, you need just run the previous sys.path... command in every new session

#path_clones = "/content/drive/MyDrive/Python_NLP/Colab_clones"
#os.chdir(path_clones)
#!git clone -b eugenefedor-patch-3 --single-branch https://github.com/eugenefedor/ipapy_eugene.git
#sys.path.append("/content/drive/MyDrive/Python_NLP/Colab_clones/ipapy_eugene")
#os.chdir(path_wd)

time: 2.01 ms (started: 2022-05-31 08:24:51 +00:00)


In [5]:
path_ipapy_clone = "/content/drive/MyDrive/Python_NLP/Colab_clones/ipapy_eugene"
sys.path.append(path_ipapy_clone)

time: 746 µs (started: 2022-05-31 08:24:51 +00:00)


In [6]:
from ipapy import UNICODE_TO_IPA
from ipapy import is_valid_ipa
from ipapy.ipachar import IPAConsonant
from ipapy.ipachar import IPAVowel
from ipapy.ipastring import IPAString

time: 2.49 s (started: 2022-05-31 08:24:51 +00:00)


In [ ]:
import pandas as pd
import re
import inspect
import numpy as np
import random

time: 263 ms (started: 2022-05-26 11:59:22 +00:00)


In [ ]:
#import custom module to populate class with wiki parsed instances 
import wiki_class
from wiki_class import Dictionary

time: 896 ms (started: 2022-05-26 11:59:22 +00:00)


In [ ]:
#unpickling instances of wiki class
import pickle
path_wiki_parsed = "/content/drive/MyDrive/Python_NLP/wiki_parsed.pickle"

#load wiki instances
with open(path_wiki_parsed, "rb") as f:
  Dictionary.instances = pickle.load(f)

time: 24.1 s (started: 2022-05-26 11:59:23 +00:00)


In [ ]:
Dictionary.get_word_from_Dict("оболочка")[0].sounds

'ɐbɐˈɫot͡ɕkə'

time: 133 ms (started: 2022-05-26 11:59:47 +00:00)


#SelectWords

In [ ]:
class SelectWords():
  @classmethod
  def set_all_attributes_to_false(cls):
    for item in Dictionary.instances:
      setattr(item, "status", False)
  
  @classmethod
  def get_number_of_items_with_true_status(cls):
    number_of_items_with_true_status = len([1 for item in Dictionary.instances if item.status])
    return number_of_items_with_true_status


  @classmethod
  def get_df_unstressed_words_lengths(cls, status = "allwords"):
    if status == "allwords":
      list_lengths_of_words = [len(item.word_lowcase) for item in Dictionary.instances]
    if status == "truewords":
      list_lengths_of_words = [len(item.word_lowcase) for item in Dictionary.instances if item.status == True]

    series_lengths_of_words = pd.Series(list_lengths_of_words, dtype = "object").value_counts()
    pd_lengths_of_unstressed_words = pd.DataFrame({"freq":series_lengths_of_words})
    pd_lengths_of_unstressed_words = pd_lengths_of_unstressed_words.reset_index()
    pd_lengths_of_unstressed_words = pd_lengths_of_unstressed_words.rename(columns = {"index": "lengths"})
    pd_lengths_of_unstressed_words = pd_lengths_of_unstressed_words.sort_values(by = "freq", ascending = True)
    pd_lengths_of_unstressed_words["percent"] = list(map(lambda x: x/len(list_lengths_of_words), 
                                              list(pd_lengths_of_unstressed_words["freq"])))
    return pd_lengths_of_unstressed_words

  @classmethod
  def get_list_of_most_frequent_unstressed_lengths(cls, threshold_persent = 0):
    pd_lengths_of_unstressed_words = cls.get_df_unstressed_words_lengths(status = "allwords")
    pd_selected = pd_lengths_of_unstressed_words[pd_lengths_of_unstressed_words["percent"] > threshold_persent]
    list_of_most_frequent_unstressed_lengths = list(pd_selected["lengths"])
    list_of_most_frequent_unstressed_lengths.sort()
    return list_of_most_frequent_unstressed_lengths

  @classmethod
  def select_unstressed_words_for_nn(cls):   
    cls.set_all_attributes_to_false()
    list_of_most_frequent_unstressed_lengths = cls.get_list_of_most_frequent_unstressed_lengths()

    pat_only_simple_words =  re.compile("[^а-я|А-Я|ё|Ё|-]")
    for item in Dictionary.instances:
      if item.word_lowcase != None \
      and re.search(pat_only_simple_words, item.word_lowcase) == None \
      and item.pos in ("noun", "verb", "adj", "name", "adv", "num","pron") \
      and len(item.word_lowcase) in list_of_most_frequent_unstressed_lengths \
      and item.accent and re.findall("'", item.accent):
      
        setattr(item, "status", True)

    return "number_of_items_with_true_status - {}".format(cls.get_number_of_items_with_true_status())

  @classmethod
  def get_rand_words_with_true_status(cls, n_of_words_to_show):
    rand_words_with_true_status = [item.word_lowcase for item in Dictionary.instances if item.status == True]
    rand_numbers = [random.randint(0, cls.get_number_of_items_with_true_status()) \
                    for i in range(n_of_words_to_show)]
    rand_words_with_true_status = [item.word_lowcase for i, item in enumerate(Dictionary.instances) \
                                  if i in rand_numbers and item.status == True
                                  ]
    return rand_words_with_true_status


time: 47.4 ms (started: 2022-05-26 11:59:47 +00:00)


In [ ]:
SelectWords.select_unstressed_words_for_nn()

'number_of_items_with_true_status - 404492'

time: 1.98 s (started: 2022-05-26 11:59:47 +00:00)


#WordSelectionByLength

In [ ]:
class WordsSelectionByLengths():
  #get df with words' lengths
  @classmethod
  def get_df_words_lengths(cls):
    number_of_true_items = len([1 for item in Dictionary.instances if item.status == True])
    list_lengths_of_words = [len(item.accent) for item in Dictionary.instances if item.status == True]
    series_lengths_of_words = pd.Series(list_lengths_of_words).value_counts()
    pd_lengths_of_words = pd.DataFrame({"freq":series_lengths_of_words})
    pd_lengths_of_words = pd_lengths_of_words.reset_index()
    pd_lengths_of_words = pd_lengths_of_words.rename(columns = {"index": "lengths"})
    pd_lengths_of_words = pd_lengths_of_words.sort_values(by = "freq", ascending = True)
    pd_lengths_of_words["percent"] = list(map(lambda x: x/number_of_true_items, 
                                              list(pd_lengths_of_words["freq"])))
    return pd_lengths_of_words

  #selecting for nn only words with length, 
  #share of which is above a defined threshold (0.01) in the total amount of words
  @classmethod
  def get_list_of_most_frequent_lengths(cls, threshold_persent):
    pd_lengths_of_words = cls.get_df_words_lengths()
    pd_selected = pd_lengths_of_words[pd_lengths_of_words["percent"] >= threshold_persent]
    list_of_selected_lengths = list(pd_selected["lengths"])
    list_of_selected_lengths.sort()
    return list_of_selected_lengths

  #setting False status to words unsatisfying condition of lengths
  @classmethod
  def unselect_words_with_low_freq_lengths(cls, threshold_persent):
    pd_lengths_of_words = cls.get_df_words_lengths()
    list_of_selected_lengths = cls.get_list_of_most_frequent_lengths(threshold_persent)
    n_of_changes = len([setattr(item, "status", False) for item in Dictionary.instances if
                      item.status == True and
                      len(item.accent) not in list_of_selected_lengths])
    return "number of unselected words is", n_of_changes


time: 20.8 ms (started: 2022-05-26 11:59:49 +00:00)


In [ ]:
WordsSelectionByLengths.get_df_words_lengths()

,lengths,freq,percent
32,36,1,0.000002
31,34,1,0.000002
30,28,2,0.000005
29,31,5,0.000012
28,32,5,0.000012
27,33,6,0.000015
26,29,9,0.000022
25,27,12,0.000030
24,2,12,0.000030
23,26,24,0.000059


time: 405 ms (started: 2022-05-26 11:59:49 +00:00)


In [ ]:
WordsSelectionByLengths.unselect_words_with_low_freq_lengths(0)

('number of unselected words is', 0)

time: 1.07 s (started: 2022-05-26 11:59:50 +00:00)


In [ ]:
WordsSelectionByLengths.get_df_words_lengths()

,lengths,freq,percent
32,36,1,0.000002
31,34,1,0.000002
30,28,2,0.000005
29,31,5,0.000012
28,32,5,0.000012
27,33,6,0.000015
26,29,9,0.000022
25,27,12,0.000030
24,2,12,0.000030
23,26,24,0.000059


time: 407 ms (started: 2022-05-26 11:59:51 +00:00)


#WordsProcessing

In [ ]:
class WordsProcessing():
  @classmethod
  def get_unique_chs_from_unstressed_words(cls):
    unique_chs_from_unstressed_words = set()
    for item in Dictionary.instances:
      if item.status:
        chs = list(item.word_lowcase)
        unique_chs_from_unstressed_words.update(chs)
    unique_chs_from_unstressed_words = list(unique_chs_from_unstressed_words)
    unique_chs_from_unstressed_words.sort()
    return unique_chs_from_unstressed_words

  @classmethod
  def get_character2number_for_unstressed_words(cls):
    unique_chs = cls.get_unique_chs_from_unstressed_words() 
    character2number_for_unstressed_words = dict((ch,i) for i, ch in enumerate(unique_chs, start = 1))
    return character2number_for_unstressed_words
  
  @classmethod
  def get_number2character_for_unstressed_words(cls): 
    unique_chs = cls.get_unique_chs_from_unstressed_words() 
    number2character_for_unstressed_words = dict((i,ch) for i, ch in enumerate(unique_chs, start = 1))
    return number2character_for_unstressed_words 

  @classmethod
  def get_max_length_of_unstressed_word(cls):
    #+1 is added as a place for accent for correct padding 
    max_length_of_unstressed_word = max([len(item.word_lowcase) \
                                        for item in Dictionary.instances if item.status == True]) + 1
    return max_length_of_unstressed_word
  

  @classmethod
  def all_unstressed_words_to_array(cls):
    #inputs_unstressed = np.zeros(max_length_of_unstressed_word)
    
    def _word2numbers(_word_lowcase):
      _numbers = []
      for ch in _word_lowcase:
        n = character2number_for_unstressed_words[ch]
        _numbers.append(n)
      _n_of_zeros_to_add = max_length_of_unstressed_word - len(_numbers) 
      _numbers.extend([0 for i in range(_n_of_zeros_to_add)])

      return _numbers

    list_of_words_as_numbers = [_word2numbers(item.word_lowcase) \
                                for item in Dictionary.instances if item.status]

    all_unstressed_words_as_array = np.vstack(list_of_words_as_numbers)
    return all_unstressed_words_as_array

time: 42.4 ms (started: 2022-05-26 11:59:51 +00:00)


In [ ]:
character2number_for_unstressed_words = WordsProcessing.get_character2number_for_unstressed_words()
print(character2number_for_unstressed_words)
number2character_for_unstressed_words = WordsProcessing.get_number2character_for_unstressed_words()
print(character2number_for_unstressed_words)
max_length_of_unstressed_word = WordsProcessing.get_max_length_of_unstressed_word()
print(max_length_of_unstressed_word)

{'-': 1, 'а': 2, 'б': 3, 'в': 4, 'г': 5, 'д': 6, 'е': 7, 'ж': 8, 'з': 9, 'и': 10, 'й': 11, 'к': 12, 'л': 13, 'м': 14, 'н': 15, 'о': 16, 'п': 17, 'р': 18, 'с': 19, 'т': 20, 'у': 21, 'ф': 22, 'х': 23, 'ц': 24, 'ч': 25, 'ш': 26, 'щ': 27, 'ъ': 28, 'ы': 29, 'ь': 30, 'э': 31, 'ю': 32, 'я': 33, 'ё': 34}
{'-': 1, 'а': 2, 'б': 3, 'в': 4, 'г': 5, 'д': 6, 'е': 7, 'ж': 8, 'з': 9, 'и': 10, 'й': 11, 'к': 12, 'л': 13, 'м': 14, 'н': 15, 'о': 16, 'п': 17, 'р': 18, 'с': 19, 'т': 20, 'у': 21, 'ф': 22, 'х': 23, 'ц': 24, 'ч': 25, 'ш': 26, 'щ': 27, 'ъ': 28, 'ы': 29, 'ь': 30, 'э': 31, 'ю': 32, 'я': 33, 'ё': 34}
36
time: 1.54 s (started: 2022-05-26 11:59:51 +00:00)


In [ ]:
all_unstressed_words_as_array = WordsProcessing.all_unstressed_words_to_array()
print(all_unstressed_words_as_array.shape)

(404492, 36)
time: 5.59 s (started: 2022-05-26 11:59:53 +00:00)


#StressProcessing

In [ ]:
class StressProcessing():
  @classmethod
  def get_position_of_stress(cls, accent):
    try: 
      position_of_stress = accent.index("'")
    except:
      print(accent)

    return position_of_stress
  
  @classmethod
  def convert_stress2array(cls, accent):
    max_length_of_unstressed_word = cls.get_max_length_of_unstressed_word()
    stress_as_array = np.zeros(max_length_of_unstressed_word)
    position_of_stress = cls.get_position_of_stress(accent)
    stress_as_array = np.insert(stress_as_array, position_of_stress, 1)
    return stress_as_array          


  @classmethod
  def all_stresses_to_array(cls):

    def _stress2array(accent):
      stress_as_array = np.zeros(max_length_of_unstressed_word)
      position_of_stress = cls.get_position_of_stress(accent)
      stress_as_array[position_of_stress] = 1
      return stress_as_array

    all_stresses_as_array = np.zeros(max_length_of_unstressed_word)

    _list_of_arrays = []
    for item in Dictionary.instances:
      if item.status == True:
        #print(item.accent)
        accent = item.accent
        position_of_stress = cls.get_position_of_stress(item.accent)
        stress_as_array = _stress2array(accent)        
        _list_of_arrays.append(stress_as_array)
    
    all_stresses_as_array = np.vstack(_list_of_arrays)
    
    return all_stresses_as_array


time: 41.8 ms (started: 2022-05-26 11:59:59 +00:00)


In [ ]:
all_stresses_as_array = StressProcessing.all_stresses_to_array()
print(all_stresses_as_array.shape)

(404492, 36)
time: 3.37 s (started: 2022-05-26 11:59:59 +00:00)


#Train_Test_Split

In [ ]:
print(all_unstressed_words_as_array.shape)
print(all_stresses_as_array.shape)

(404492, 36)
(404492, 36)
time: 1.39 ms (started: 2022-05-26 12:00:02 +00:00)


In [ ]:
def split_data_by_ratio(split_ratio, all_unstressed_words_as_array, all_stresses_as_array):
  np.random.seed(30)
  number_of_items_with_true_status = SelectWords.get_number_of_items_with_true_status()
  
  mask = np.random.rand(number_of_items_with_true_status) < split_ratio
  
  inputs = all_unstressed_words_as_array[mask]
  inputs_test = all_unstressed_words_as_array[~mask]
  
  labels = all_stresses_as_array[mask]
  labels_test = all_stresses_as_array[~mask]

  return inputs, labels, inputs_test, labels_test

inputs, labels, inputs_test, labels_test = split_data_by_ratio(0.8, all_unstressed_words_as_array, all_stresses_as_array)
print("inputs",inputs.shape)
print("labels",labels.shape)
print("inputs_test",inputs_test.shape)
print("labels_test",labels_test.shape)

inputs (323462, 36)
labels (323462, 36)
inputs_test (81030, 36)
labels_test (81030, 36)
time: 321 ms (started: 2022-05-26 12:00:02 +00:00)


#Model

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.layers import GRU, LSTM, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding

time: 3.17 s (started: 2022-05-26 12:00:02 +00:00)


In [ ]:
print("lables.shape",labels.shape)
print("inputs.shape",inputs.shape)
print("max_length_of_unstressed_word",max_length_of_unstressed_word)
size_of_vocab = len(character2number_for_unstressed_words)+1
print("size_of_vocab", size_of_vocab)

lables.shape (323462, 36)
inputs.shape (323462, 36)
max_length_of_unstressed_word 36
size_of_vocab 35
time: 3.48 ms (started: 2022-05-26 12:00:06 +00:00)


In [ ]:
def desigh_model_stress(inputs, size_of_vocab, max_length_of_unstressed_word):
  inputs_stress_keras = tf.keras.Input(shape = (inputs.shape[1],))

  embd_stress = tf.keras.layers.Embedding(size_of_vocab, 16)
  embd_stress_result = embd_stress(inputs_stress_keras)

  my_gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, dropout = 0.3, recurrent_dropout = 0.3))
  result_my_gru = my_gru(embd_stress_result)
  
  dns_stress = tf.keras.layers.Dense(max_length_of_unstressed_word, activation = "softmax")

  outputs_stress = dns_stress(result_my_gru)

  model_stress = tf.keras.Model(inputs = inputs_stress_keras, outputs = outputs_stress) 
  return model_stress

model_stress = desigh_model_stress(inputs, size_of_vocab, max_length_of_unstressed_word)

time: 3.75 s (started: 2022-05-26 12:00:06 +00:00)


In [ ]:
model_stress.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 36)]              0         
                                                                 
 embedding (Embedding)       (None, 36, 16)            560       
                                                                 
 bidirectional (Bidirectiona  (None, 64)               9600      
 l)                                                              
                                                                 
 dense (Dense)               (None, 36)                2340      
                                                                 
Total params: 12,500
Trainable params: 12,500
Non-trainable params: 0
_________________________________________________________________
time: 57.7 ms (started: 2022-05-26 12:00:09 +00:00)


In [ ]:
filepath_stress="/content/drive/MyDrive/Python_NLP/checkpoints/checkpoint_put_stress.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath_stress, 
                                                monitor='categorical_accuracy', 
                                                verbose=1, 
                                                save_best_only=True, 
                                                mode='max')

time: 2.45 ms (started: 2022-05-26 12:00:09 +00:00)


In [ ]:
model_stress.compile(loss = keras.losses.CategoricalCrossentropy(), 
              optimizer = keras.optimizers.Adam(learning_rate=0.001),
              metrics = "categorical_accuracy")

time: 33.4 ms (started: 2022-05-26 12:00:09 +00:00)


In [ ]:
#model_stress.load_weights(filepath_stress)

time: 772 µs (started: 2022-05-26 12:00:09 +00:00)


In [ ]:
'''
history_stress = model_stress.fit(inputs, labels, 
                    batch_size=64, 
                    epochs=100, 
                    #validation_split=0.2, 
                    verbose=1, 
                    callbacks=[checkpoint],
                    )
'''

'\nhistory_stress = model_stress.fit(inputs, labels, \n                    batch_size=64, \n                    epochs=100, \n                    #validation_split=0.2, \n                    verbose=1, \n                    callbacks=[checkpoint],\n                    )\n'

time: 4.65 ms (started: 2022-05-26 12:00:09 +00:00)


In [ ]:
#save entire model
#from tensorflow.keras.models import load_model
path_model_stress = '/content/drive/MyDrive/Python_NLP/checkpoints/Model_put_stress' 
#model_stress.save(path_model_stress)  

time: 1.37 ms (started: 2022-05-26 12:00:09 +00:00)


#Load Model

In [ ]:
#load model
del model_stress
from tensorflow.keras.models import load_model
model_stress = load_model(path_model_stress)

time: 5.56 s (started: 2022-05-26 12:00:10 +00:00)


#Predict

In [ ]:
model_stress.evaluate(inputs_test, labels_test)

2533/2533 [==============================] - 56s 22ms/step - loss: 0.4989 - categorical_accuracy: 0.7945


[0.4989108145236969, 0.7944958806037903]

time: 55.8 s (started: 2022-05-26 12:04:08 +00:00)


In [ ]:
def put_stress(word_lowcase):

  def _word2numbers(word_lowcase):
    _numbers = []
    for ch in word_lowcase:
      n = character2number_for_unstressed_words[ch]
      _numbers.append(n)
    _n_of_zeros_to_add = max_length_of_unstressed_word - len(_numbers) 
    _numbers.extend([0 for i in range(_n_of_zeros_to_add)])
    _numbers = np.array(_numbers)
    _numbers = _numbers.reshape((1,max_length_of_unstressed_word))
    return _numbers

  stress_as_array = model_stress.predict(_word2numbers(word_lowcase))
  
  def numbers2word_stressed(stress_as_array):
    index_of_stress = np.argmax(stress_as_array)
    word_stressed = word_lowcase[:index_of_stress] + "'" + word_lowcase[index_of_stress:]
    return word_stressed
  
  word_stressed = numbers2word_stressed(stress_as_array)
  return  word_stressed

time: 11.2 ms (started: 2022-05-26 12:05:04 +00:00)


In [ ]:
put_stress("булочка")

"бу'лочка"

time: 562 ms (started: 2022-05-26 12:05:04 +00:00)


In [ ]:
def put_stress_comparison_dict_predict(word_unstressed):
  word_stressed_predicted = put_stress(word_unstressed)
  #print("predicted word stressed", word_stressed_predicted)
  #print(word_unstressed)
  try:
    word_stressed_from_dict =  Dictionary.get_word_from_Dict(word_unstressed)[0].accent
    #print("dictionary word stressed",  word_stressed_from_dict)
  except:
    word_stressed_from_dict = "no word in dict" 
    #print("no word in dict")
  
  stress_comparison = "word_stressed_predicted - {},\n \
  word_stressed_from_dict - {},\n".\
  format(word_stressed_predicted, word_stressed_from_dict)  

  return print(stress_comparison)

time: 5.19 ms (started: 2022-05-26 12:05:04 +00:00)


In [ ]:
put_stress_comparison_dict_predict("дума")

word_stressed_predicted - ду'ма,
   word_stressed_from_dict - ду'ма,

time: 208 ms (started: 2022-05-26 12:05:04 +00:00)
